<a href="https://colab.research.google.com/github/WillA656/NLP_Project/blob/main/SparkNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's set up SparkNLP.

## Hello world -- Part 2

In [1]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2024-07-24 23:40:38--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 3.86.22.73
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|3.86.22.73|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2024-07-24 23:40:38--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2024-07-24 23:40:38 (27.2 MB/s) - written to stdout [1191/1191]

Installing PySpark

In [2]:
# Access Data
%%capture
!curl "https://raw.githubusercontent.com/WillA656/NLP_Project/main/Gutendex_JSON" -o book_list
!wget -i book_list

# ! rm *.utf-8.* # remove all utf

In [3]:
# Rename files
import os
os.mkdir("book_data")
for filename in os.listdir("."):
  if filename.endswith("utf-8"):
    with open(filename) as f:
      first_line = f.readline()
      print(first_line[32:])
    os.rename(filename, (f"book_data/{first_line[32:]}").rstrip()+".txt") # rename and move to new file

A Room with a View

Romeo and Juliet

Great Expectations

Middlemarch

The Blue Castle: a novel

Little Women; Or, Meg, Jo, Beth, and Amy

The Enchanted April

Pride and Prejudice

The Adventures of Ferdinand Count Fathom — Complete

Moby Dick; Or, The Whale



In [4]:
#Combine all 10 books into our corpus (Requires titles to end in .txt)
output_path = '/content/corpus.txt'
with open(output_path, 'w') as out:
    for filename in os.listdir('book_data'):
        file_path = os.path.join('book_data', filename)
        if filename.endswith('.txt'):
            with open(file_path, 'r') as infile:
                out.write(infile.read() + '\n')

In [5]:
!pip install pyspark
!pip install sparknlp
import sparknlp
spark = sparknlp.start()

from sparknlp.pretrained import PretrainedPipeline

In [6]:
pipeline = PretrainedPipeline("explain_document_ml")

explain_document_ml download started this may take some time.
Approx size to download 9 MB
[OK!]


In [7]:
from pyspark.sql.functions import col, expr, explode, arrays_zip
df = spark.read.text("/content/corpus.txt")
df = df.withColumnRenamed("value", "text")
result = pipeline.transform(df)
result = result.select(
    col("token.result").alias("token_result"),
    col("pos.result").alias("pos_result")
)
result.show()

+--------------------+--------------------+
|        token_result|          pos_result|
+--------------------+--------------------+
|[The, Project, Gu...|[DT, NNP, NNP, NN...|
|                  []|                  []|
|[This, ebook, is,...|[DT, NN, VBZ, IN,...|
|[most, other, par...|[JJS, JJ, NNS, IN...|
|[whatsoever, ., Y...|[RB, ., PRP, MD, ...|
|[of, the, Project...|[IN, DT, NNP, NNP...|
|[at, www.gutenber...|[IN, NN, ., IN, P...|
|[you, will, have,...|[PRP, MD, VB, TO,...|
|[before, using, t...|[IN, VBG, DT, NN, .]|
|                  []|                  []|
|[Title, :, Great,...|  [NNP, :, NNP, NNP]|
|                  []|                  []|
|[Author, :, Charl...|   [NN, :, NNP, NNP]|
|                  []|                  []|
|[Release, date, :...|[NNP, NN, :, NNP,...|
|[Most, recently, ...|[JJS, RB, VBN, :,...|
|                  []|                  []|
|[Language, :, Eng...|       [NNP, :, NNP]|
|                  []|                  []|
|[Credits, :, An, ...|[NNS, :, D

We can use some recent headlines.

In [8]:
result.printSchema()

tokens_and_pos = result.select(explode(expr("arrays_zip(token_result, pos_result)")).alias("zipped"))

tokens_and_pos_split = tokens_and_pos.selectExpr(
    "zipped.token_result as token",
    "zipped.pos_result as pos"
)
tokens_and_pos_split.show()

root
 |-- token_result: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- pos_result: array (nullable = true)
 |    |-- element: string (containsNull = true)

+------------+----+
|       token| pos|
+------------+----+
|         The|  DT|
|     Project| NNP|
|   Gutenberg| NNP|
|       eBook|  NN|
|          of|  IN|
|       Great| NNP|
|Expectations| NNP|
|        This|  DT|
|       ebook|  NN|
|          is| VBZ|
|         for|  IN|
|         the|  DT|
|         use|  NN|
|          of|  IN|
|      anyone|  NN|
|    anywhere|  RB|
|          in|  IN|
|         the|  DT|
|      United| NNP|
|      States|NNPS|
+------------+----+
only showing top 20 rows



In [9]:
pronoun_tags = ["PRP", "PRP$"]
pronouns_df = tokens_and_pos_split.filter(col("pos").isin(pronoun_tags))
pronouns_df.show()

+------+----+
| token| pos|
+------+----+
|   You| PRP|
|    it| PRP|
|    it| PRP|
|    it| PRP|
|   you| PRP|
|   you| PRP|
|   you| PRP|
|     I| PRP|
|     I| PRP|
|    My|PRP$|
|    my|PRP$|
|    my|PRP$|
|     I| PRP|
|myself| PRP|
|     I| PRP|
|    my|PRP$|
|   his|PRP$|
|    my|PRP$|
|     I| PRP|
|    my|PRP$|
+------+----+
only showing top 20 rows



In [ ]:
gendered_pronouns = ["he", "him", "his", "himself", "she", "her", "hers", "herself", "they", "them", "their", "theirs"]

gendered_pronouns_df = pronouns_df.filter(col("token").isin(gendered_pronouns))
gendered_pronouns_df.show()

len(gendered_pronouns_df.collect())

+-----+----+
|token| pos|
+-----+----+
|  his|PRP$|
| them| PRP|
|their|PRP$|
| they| PRP|
|their|PRP$|
|   he| PRP|
|their|PRP$|
| they| PRP|
|their|PRP$|
|their|PRP$|
|their|PRP$|
| them| PRP|
|  his|PRP$|
|  his|PRP$|
|  his|PRP$|
|   he| PRP|
| them| PRP|
|   he| PRP|
|   he| PRP|
|   he| PRP|
+-----+----+
only showing top 20 rows



In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
contingency_df = gendered_pronouns_df.groupBy("token", "pos").agg(F.count("*").alias("count"))

contingency_df.show()

In [ ]:
from scipy.stats import chi2_contingency
import pandas as pd

# Convert the Spark DataFrame to a Pandas DataFrame for ease of use
contingency_pd = contingency_df.toPandas()

# Pivot the DataFrame to get a matrix form (tokens as rows, pos as columns)
pivot_table = contingency_pd.pivot(index='token', columns='pos', values='count').fillna(0)

# Perform the chi-squared test
chi2, p, dof, expected = chi2_contingency(pivot_table)

# Display the results
print("Chi-Squared Statistic:", chi2)
print("p-value:", p)
print("Degrees of Freedom:", dof)
print("Expected Frequencies:\n", expected)